In [0]:
%pip install snowflake-connector-python
%pip install snowflake-snowpark-python

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
%restart_python

In [0]:
import pandas as pd
import os
from snowflake.connector import connect
from snowflake.connector.pandas_tools import write_pandas
from snowflake.snowpark import Session
from snowflake.snowpark.functions import col, to_date, upper, lit, monthname, year, model
from snowflake.snowpark import functions as Fun


In [0]:
# 1. Import Snowpark Session
from snowflake.snowpark import Session

# 2. Connection parameters
connection_parameters = {
  "account": "ewkbimp-kg51208",
  "user": "THARANI",
  "password": "TharaniMugunthan13",
  "warehouse": "COMPUTE_WH",
  "database": "RETAIL",
  "schema": "RAW_SALES"
}

# 3. Create session
session = Session.builder.configs(connection_parameters).create()

print("Connected to Snowflake successfully!")


Connected to Snowflake successfully!


In [0]:
df_raw = session.table("SALES")

In [0]:
raw_sales_df.show(5)

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"ORDERID"     |"ORDERDATE"  |"MONTHOFSALE"  |"CUSTOMERID"  |"CUSTOMERNAME"  |"COUNTRY"  |"REGION"  |"CITY"   |"CATEGORY"       |"SUBCATEGORY"  |"QUANTITY"  |"DISCOUNT"  |"SALES"  |"PROFIT"  |
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|ORD-5F8D6F0C  |2024-10-08   |2024-10        |CUST1000      |Ananya Sharma   |India      |South     |Mumbai   |Office Supplies  |Paper          |9           |0.0         |2700.0   |780.43    |
|ORD-BF0078E4  |2024-08-11   |2024-08        |CUST1001      |Aarav Iyer      |India      |Central   |Lucknow  |Technology       |Networking     |4           |0.15        |27200.0  |4135.6    |
|ORD-86CD58A3  |2024-06-12   |2024-

In [0]:
# Convert and clean data 
df_clean = (
    df_raw
    .withColumn("OrderDate", to_date(col("OrderDate"), "yyyy-MM-dd"))  # Fixed format string
    .withColumn("Country", upper(col("Country")))
    .withColumn("City", upper(col("City")))
    .withColumn("Category", upper(col("Category")))
    .withColumn("Profit", col("Profit").cast("float"))
    .withColumn("Sales", col("Sales").cast("float"))
    .withColumn("Discount", col("Discount").cast("float"))
    .withColumn("Quantity", col("Quantity").cast("int"))
)

# Verify the transformation
print("Sample of cleaned data:")
df_clean.select("OrderID", "OrderDate", "Country", "Sales", "Profit").show(5)


Sample of cleaned data:
---------------------------------------------------------------
|"ORDERID"     |"ORDERDATE"  |"COUNTRY"  |"SALES"  |"PROFIT"  |
---------------------------------------------------------------
|ORD-5F8D6F0C  |2024-10-08   |INDIA      |2700.0   |780.43    |
|ORD-BF0078E4  |2024-08-11   |INDIA      |27200.0  |4135.6    |
|ORD-86CD58A3  |2024-06-12   |USA        |31500.0  |5676.96   |
|ORD-FB0CD2D9  |2024-12-18   |INDIA      |36000.0  |11783.22  |
|ORD-EF35596B  |2024-10-27   |UK         |24000.0  |4189.98   |
---------------------------------------------------------------



In [0]:
df_clean.write.mode("overwrite").save_as_table("FACT_SALES")